In [2]:
from google.colab import drive
drive.mount('/content/drive')

!cp drive/MyDrive/optimizers.py .
!cp drive/MyDrive/yellowfin.py .

from optimizers import DemonRanger
from yellowfin import YFOptimizer

from sklearn.model_selection import KFold
import sklearn
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torchvision
from torch.nn import functional as F

import numpy as np
import time 

import itertools
from tqdm.notebook import tqdm

from torch.optim.lr_scheduler import StepLR

import matplotlib.pyplot as plt

import torchvision.datasets as datasets

!pip install CosineAnnealingWithRestartsLR
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import LinearLR
from torch.optim.lr_scheduler import ExponentialLR

! [ ! -z "$COLAB_GPU" ] && pip install torch scikit-learn==0.20.* skorch

torch.manual_seed(42)

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement CosineAnnealingWithRestartsLR (from versions: none)
ERROR: No matching distribution found for CosineAnnealingWithRestartsLR
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.4 MB 1.2 MB/s 
     |████████████████████████████████| 155 kB 69.3 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.20.4 which is incompatible.
imbalanced-learn 0.8.

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
      # print(f'Reset trainable parameters of layer = {layer}')
      layer.reset_parameters()

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.linear1 = nn.Linear(28*28, 256) 
        self.linear2 = nn.Linear(256, 100) 
        self.final = nn.Linear(100, 10)
        self.relu = nn.ReLU()

    def forward(self, img):
        x = img.view(-1, 28*28)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.final(x)
        return x

In [6]:
def train(model, data_loader, criterion, optimizer, mini_batch_size, view_every = 500):
    running_loss = 0.0

    for i, data in enumerate(data_loader, 0):
        
        # get the input data
        inputs = data[0].to(device)
        labels = data[1].to(device)
        
        # zero the gradients
        optimizer.zero_grad()

        # forward + backward + optimizte
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % view_every == view_every-1: 
            print('Batch {:d} / {:d}: loss = {:.4f}'.format(i+1,len(data_loader),running_loss/view_every))
            running_loss = 0.0

In [7]:
def test(model, data_loader):
    correct = 0.0
    total = 0.0
        
    with torch.no_grad():
      for data in data_loader:
        images = data[0].to(device)
        labels = data[1].to(device)
        
        # compute outputs by running images through the network
        outputs = model(images)
        
        # choosing class with highest probability          
        pdf = F.softmax(outputs, dim=1) 
        predicted = pdf.argmax(dim=1)     

        correct += (predicted ==  labels).sum()
        total += len(predicted)
    return correct/total 

In [8]:
def hyperparameters(model, params):
  start_time = time.time()
  keys, values = zip(*params.items())
  permutations_dict = [dict(zip(keys, v)) for v in itertools.product(*values)]

  opt_acc = 0.0
  opt_config = {}

  for config in tqdm(permutations_dict):
    print('-----------------------')
    print("Current Parameters: ", config)
    acc = cross_validation_accuracy(model, config)

    if acc > opt_acc:
      opt_acc = acc
      opt_config = config
  
  end_time = time.time()
  print(f" Best Accuracy: {opt_acc}, with parameters {opt_config}")
  print(f" Runtime: {((end_time - start_time)/60):.4} minutes")
  return opt_config, opt_acc

In [9]:
def cross_validation_accuracy(model, params):
  # Define the K-fold Cross Validator
  kfold = KFold(n_splits = params["folds"], shuffle = True)

  # dict to save results
  accuracies = {}

  for fold, (train_idx, val_idx) in enumerate(kfold.split(mnist_train)):
    print(f"-------------Fold {fold+1}----------------")

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

    # Define the data loaders for training and testing data in this fold
    train_loader = torch.utils.data.DataLoader(mnist_train, 
                                               batch_size = params["batch_size"], 
                                               sampler = train_subsampler, 
                                               pin_memory = torch.cuda.is_available())
    val_loader = torch.utils.data.DataLoader(mnist_train, 
                                             batch_size = params["batch_size"], 
                                             sampler = val_subsampler, 
                                             pin_memory = torch.cuda.is_available())

    # Initialize the neural network
    model.apply(reset_weights)

    # Initialize model's parameters
    criterion = params["criterion"]()
    optimizer = params["optimizer"](model.parameters(), lr = params["lr"])
    scheduler = params["scheduler"](optimizer, **params["scheduler_parameters"])

    val_acc = 0.0
    for epoch in range(params["epochs"]):
      train(model, train_loader, criterion, optimizer, params["batch_size"])
      curr_acc = (test(model, val_loader)) * 100.0
      print(f"epoch {epoch + 1}/{params['epochs']}: Accuracy: {curr_acc:.4}")
      val_acc = curr_acc

      try:
        scheduler.step()
      except:
        scheduler.step(curr_acc)

    accuracies[fold] = curr_acc

  # Print fold results
  print('--------------------------------')
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {params["folds"]} FOLDS')
  print('--------------------------------')

  sum = 0.0
  for fold, value in accuracies.items():
    print(f"Fold {fold}: {value} %")
    sum += value
  
  avg_acc = sum/len(accuracies.items())
  print(f"Average: {avg_acc:.4} %")
  print("-----------")
  
  return avg_acc

### Data & Loaders

In [10]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

mnist_train = datasets.MNIST(root='./data', 
                             train=True, 
                             download=True, 
                             transform=transform)
mnist_test = torchvision.datasets.MNIST('./data/files/', 
                                        train=False, 
                                        download=True,  
                                        transform=transform)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True, pin_memory=torch.cuda.is_available())
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=64, shuffle=True, pin_memory = torch.cuda.is_available())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Extracting ./data/files/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/files/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/files/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/files/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/files/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/files/MNIST/raw



In [11]:
mnist_train.data.shape, mnist_test.data.shape

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

## Finding Hyperparameters for each Model


### LambdaLR

In [12]:
# def lambda1(epoch):
#   return 1  
def lambda2(epoch):
  return epoch // 30
def lambda3(epoch):
  return 0.95 ** epoch 

parameters = {
    "criterion": [torch.nn.CrossEntropyLoss],
    "lr": [0.002, 0.02, 0.2],
    "optimizer": [torch.optim.Adam],
    "batch_size": [90, 120, 180],
    "folds": [3],
    "epochs": [25],
    "scheduler": [LambdaLR],
    "scheduler_parameters": [{"lr_lambda" : lambda2}, 
                             {"lr_lambda" : lambda3}]}

model = Net()
model.to(device)

opt_config_lambda, opt_acc_lambda = hyperparameters(model, parameters)

  0%|          | 0/18 [00:00<?, ?it/s]

-----------------------
Current Parameters:  {'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'lr': 0.002, 'optimizer': <class 'torch.optim.adam.Adam'>, 'batch_size': 90, 'folds': 3, 'epochs': 25, 'scheduler': <class 'torch.optim.lr_scheduler.LambdaLR'>, 'scheduler_parameters': {'lr_lambda': <function lambda2 at 0x7f885d88b5f0>}}
-------------Fold 1----------------
epoch 1/25: Accuracy: 8.325
epoch 2/25: Accuracy: 8.325
epoch 3/25: Accuracy: 8.325
epoch 4/25: Accuracy: 8.325
epoch 5/25: Accuracy: 8.325
epoch 6/25: Accuracy: 8.325
epoch 7/25: Accuracy: 8.325
epoch 8/25: Accuracy: 8.325
epoch 9/25: Accuracy: 8.325
epoch 10/25: Accuracy: 8.325
epoch 11/25: Accuracy: 8.325
epoch 12/25: Accuracy: 8.325
epoch 13/25: Accuracy: 8.325
epoch 14/25: Accuracy: 8.325
epoch 15/25: Accuracy: 8.325
epoch 16/25: Accuracy: 8.325
epoch 17/25: Accuracy: 8.325
epoch 18/25: Accuracy: 8.325
epoch 19/25: Accuracy: 8.325
epoch 20/25: Accuracy: 8.325
epoch 21/25: Accuracy: 8.325
epoch 22/25: Accu

### CosineAnnealingLR

In [12]:
parameters = {
    "criterion": [torch.nn.CrossEntropyLoss],
    "lr": [0.002, 0.02],
    "optimizer": [torch.optim.Adam],
    "batch_size": [90, 120],
    "folds": [3],
    "epochs": [25],
    "scheduler": [CosineAnnealingLR],
    "scheduler_parameters": [{"T_max" : 3, "eta_min": 0.02}, 
                             {"T_max" : 5, "eta_min": 0.02}, 
                             {"T_max" : 8, "eta_min": 0.02}]}  

model = Net()
model.to(device)

opt_config_cos, opt_acc_cos = hyperparameters(model, parameters)


  0%|          | 0/12 [00:00<?, ?it/s]

-----------------------
Current Parameters:  {'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'lr': 0.002, 'optimizer': <class 'torch.optim.adam.Adam'>, 'batch_size': 90, 'folds': 3, 'epochs': 25, 'scheduler': <class 'torch.optim.lr_scheduler.CosineAnnealingLR'>, 'scheduler_parameters': {'T_max': 3, 'eta_min': 0.02}}
-------------Fold 1----------------
epoch 1/25: Accuracy: 94.9
epoch 2/25: Accuracy: 95.9
epoch 3/25: Accuracy: 94.03
epoch 4/25: Accuracy: 94.05
epoch 5/25: Accuracy: 95.46
epoch 6/25: Accuracy: 97.0
epoch 7/25: Accuracy: 97.25
epoch 8/25: Accuracy: 97.07
epoch 9/25: Accuracy: 94.7
epoch 10/25: Accuracy: 93.7
epoch 11/25: Accuracy: 96.15
epoch 12/25: Accuracy: 96.86
epoch 13/25: Accuracy: 97.01
epoch 14/25: Accuracy: 96.83
epoch 15/25: Accuracy: 94.98
epoch 16/25: Accuracy: 93.47
epoch 17/25: Accuracy: 96.03
epoch 18/25: Accuracy: 96.83
epoch 19/25: Accuracy: 97.11
epoch 20/25: Accuracy: 97.02
epoch 21/25: Accuracy: 95.05
epoch 22/25: Accuracy: 95.11
epoch 

### LinearLR

In [ ]:
parameters = {
    "criterion": [torch.nn.CrossEntropyLoss],
    "lr": [0.002, 0.05, 0.2],
    "optimizer": [torch.optim.Adam],
    "batch_size": [90, 120, 180],
    "folds": [5],
    "epochs": [25],
    "scheduler": [LinearLR],
    "scheduler_parameters": [{"start_factor":0.01}, {"start_factor": 0.03}, {"start_factor": 0.05}],
   }

model = Net()
model.to(device)

opt_config_linear, opt_acc_linear = hyperparameters(model, parameters)

  0%|          | 0/27 [00:00<?, ?it/s]

Current Parameters:  {'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'lr': 0.002, 'optimizer': <class 'torch.optim.adam.Adam'>, 'batch_size': 90, 'folds': 5, 'epochs': 25, 'scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_parameters': {'start_factor': 0.01}}
Fold  1
device being used:  cuda
epoch 1/25, Validation Accuracy: 80.62
epoch 2/25, Validation Accuracy: 94.32
epoch 3/25, Validation Accuracy: 96.36
epoch 4/25, Validation Accuracy: 96.98
epoch 5/25, Validation Accuracy: 97.3
epoch 6/25, Validation Accuracy: 97.14
epoch 7/25, Validation Accuracy: 97.44
epoch 8/25, Validation Accuracy: 97.52
epoch 9/25, Validation Accuracy: 97.38
epoch 10/25, Validation Accuracy: 97.69
epoch 11/25, Validation Accuracy: 97.29
epoch 12/25, Validation Accuracy: 97.64
epoch 13/25, Validation Accuracy: 97.48
epoch 14/25, Validation Accuracy: 97.32
epoch 15/25, Validation Accuracy: 97.58
epoch 16/25, Validation Accuracy: 97.7
epoch 17/25, Validation Accuracy: 97.45
epoc

### ExponentialLR

In [ ]:
parameters = {
    "criterion": [torch.nn.CrossEntropyLoss],
    "lr": [0.002, 0.05, 0.2],
    "optimizer": [torch.optim.Adam],
    "batch_size": [90, 120, 180],
    "folds": [5],
    "epochs": [25],
    "scheduler": [ExponentialLR],
    "scheduler_parameters": [{"gamma":0.01}, {"gamma": 0.03}, {"gamma": 0.05}],
   }

model = Net()
model.to(device)

opt_config_exp, opt_acc_exp = hyperparameters(model, parameters)

  0%|          | 0/27 [00:00<?, ?it/s]

Current Parameters:  {'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'lr': 0.002, 'optimizer': <class 'torch.optim.adam.Adam'>, 'batch_size': 90, 'folds': 5, 'epochs': 25, 'scheduler': <class 'torch.optim.lr_scheduler.ExponentialLR'>, 'scheduler_parameters': {'gamma': 0.01}}
Fold  1
device being used:  cuda
epoch 1/25, Validation Accuracy: 96.39
epoch 2/25, Validation Accuracy: 96.87
epoch 3/25, Validation Accuracy: 96.86
epoch 4/25, Validation Accuracy: 96.86
epoch 5/25, Validation Accuracy: 96.86
epoch 6/25, Validation Accuracy: 96.86
epoch 7/25, Validation Accuracy: 96.86
epoch 8/25, Validation Accuracy: 96.86
epoch 9/25, Validation Accuracy: 96.86
epoch 10/25, Validation Accuracy: 96.86
epoch 11/25, Validation Accuracy: 96.86
epoch 12/25, Validation Accuracy: 96.86
epoch 13/25, Validation Accuracy: 96.86
epoch 14/25, Validation Accuracy: 96.86
epoch 15/25, Validation Accuracy: 96.86
epoch 16/25, Validation Accuracy: 96.86
epoch 17/25, Validation Accuracy: 96.86
epoc

## ADAM Demon

In [24]:
def get_demon(model, config, train_loader):
  demon_optimizer = DemonRanger(params = model.parameters(),
                        lr = config["lr"],
                        weight_decay = config["wd"],
                        epochs = config["epochs"],
                        step_per_epoch = len(train_loader), 
                        betas = (0.9,0.999,0.999), # restore default AdamW betas
                        nus = (1.0,1.0), # disables QHMomentum
                        k = 0,  # disables lookahead
                        alpha = 1.0, 
                        IA = False, # enables Iterate Averaging
                        rectify = False, # disables RAdam Recitification
                        AdaMod = False, #disables AdaMod
                        AdaMod_bias_correct = False, #disables AdaMod bias corretion (not used originally)
                        use_demon = True, #enables Decaying Momentum (DEMON)
                        use_gc = False, #disables gradient centralization
                        amsgrad = False # disables amsgrad
                       )
  return demon_optimizer

In [29]:
def train_demon(model, train_loader, criterion, optimizer, view_every, IA_activate = False):
  running_loss = 0.0
        
  for i, data in enumerate(train_loader, 0):
      # get the inputs; data is a list of [inputs, labels]
      inputs, labels = data[0].to(device), data[1].to(device)

      # zero the parameter gradients
      optimizer.zero_grad()

      # forward + backward + optimize
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()

      # val_acc = curr_acc
      optimizer.step(IA_activate)

  # print('Finished Training')

In [30]:
def test_demon(model, test_loader):
  correct = 0
  total = 0
  # since we're not training, we don't need to calculate the gradients for our outputs
  with torch.no_grad():
      for data in test_loader:
          images, labels = data[0].to(device), data[1].to(device)
          # calculate outputs by running images through the network 
          outputs = model(images).to(device)
          
          # the class with the highest energy is what we choose as prediction
          pdf = F.softmax(outputs, dim=1) 
          predicted = pdf.argmax(dim=1) 

          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  # print(f'Accuracy of the network on the test images: {(100 * correct / total)} %')
  return correct/total

In [31]:
def hyperparameters_demon(model, params):
  start_time = time.time()
  keys, values = zip(*params.items())
  permutations_dict = [dict(zip(keys, v)) for v in itertools.product(*values)]

  opt_acc = 0.0
  opt_config = {}

  for config in tqdm(permutations_dict):
    print('-----------------------')
    print("Current Parameters: ", config)
    acc = cross_validation_demon(model, config)

    if acc > opt_acc:
      opt_acc = acc
      opt_config = config
  
  end_time = time.time()
  print(f" Best Accuracy: {opt_acc}, with parameters {opt_config}")
  print(f" Runtime: {((end_time - start_time)/60):.4} minutes")
  return opt_config, opt_acc

In [32]:
def cross_validation_demon(model, params, IA_activate = False):
  kfold = KFold(n_splits = params["folds"], shuffle = True)
  results = {}

  for fold, (train_idx, val_idx) in enumerate(kfold.split(mnist_train)):
    print(f"---------------Fold {fold+1}--------------")

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

    train_loader = torch.utils.data.DataLoader(mnist_train, 
                                               batch_size=params["batch_size"],
                                               sampler = train_subsampler,
                                               pin_memory=torch.cuda.is_available())
    val_loader = torch.utils.data.DataLoader(mnist_train, 
                                             batch_size=params["batch_size"],
                                             sampler = val_subsampler,
                                             pin_memory = torch.cuda.is_available())
    
    demon = get_demon(model, params, train_loader)

    model.apply(reset_weights)

    val_acc = 0.0
    for epoch in range(params["epochs"]):
      train_demon(model, train_loader, params["criterion"](), demon, params["view_every"])
      curr_acc = (test_demon(model, val_loader)) * 100.0
      val_acc = curr_acc

      print(f"epoch {epoch + 1}/{params['epochs']}, Accuracy: {curr_acc:.4}")
    
    results[fold] = curr_acc

  # Print fold results
  print('--------------------------------')
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {params["folds"]} FOLDS')
  print('--------------------------------')

  sum = 0.0
  for fold, value in results.items():
    print(f"Fold {fold}: {value} %")
    sum += value
  
  avg_acc = sum/len(results.items())
  print(f"Average: {avg_acc:.4} %")
  print("-----------")
  
  return avg_acc

In [ ]:
parameters_demon = {
    "criterion": [torch.nn.CrossEntropyLoss],
    "lr": [0.002, 0.02, 0.2],
    "batch_size": [90, 120, 180],
    "folds": [5],
    "epochs": [25],
    "wd": [0],
    "view_every": [1000]
   }

model = Net()
model.to(device)

opt_config_demon, opt_acc_demon = hyperparameters_demon(model, parameters_demon)

  0%|          | 0/9 [00:00<?, ?it/s]

Current Parameters:  {'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'lr': 0.002, 'batch_size': 90, 'folds': 5, 'epochs': 25, 'wd': 0, 'view_every': 1000}
---------------Fold 1--------------
device being used:  cuda


RuntimeError: ignored

## YellowFin

In [11]:
def get_yellowfin(model, config):
  yellowfin_optimizer = YFOptimizer(model.parameters(),
                        lr = config["lr"],
                        weight_decay = config["wd"],
                        mu = config["mu"])
  return yellowfin_optimizer

In [15]:
def hyperparameters_yellowfin(model, params, device):
  start_time = time.time()
  keys, values = zip(*params.items())
  permutations_dict = [dict(zip(keys, v)) for v in itertools.product(*values)]

  opt_acc = 0.0
  opt_config = {}

  for config in tqdm(permutations_dict):
    print('-----------------------')
    print("Current Parameters: ", config)
    acc = cross_validation_yellowfin(model, config, get_yellowfin(model, config), device)

    if acc > opt_acc:
      opt_acc = acc
      opt_config = config
  
  end_time = time.time()
  print(f" Best Accuracy: {opt_acc}, with parameters {opt_config}")
  print(f" Runtime: {((end_time - start_time)/60):.4} minutes")
  return opt_config, opt_acc

In [19]:
def cross_validation_yellowfin(model, params, yellowfin, device, IA_activate = False):
  kfold = KFold(n_splits = params["folds"], shuffle = True)
  results = {}

  for fold, (train_idx, val_idx) in enumerate(kfold.split(mnist_train)):
    print(f"---------------Fold {fold+1}--------------")

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

    train_loader = torch.utils.data.DataLoader(mnist_train, 
                                               batch_size=params["batch_size"],
                                               sampler = train_subsampler,
                                               pin_memory=torch.cuda.is_available())
    val_loader = torch.utils.data.DataLoader(mnist_train, 
                                             batch_size=params["batch_size"],
                                             sampler = val_subsampler,
                                             pin_memory = torch.cuda.is_available())

    model.apply(reset_weights)

    val_acc = 0.0
    for epoch in range(params["epochs"]):
      train(model, train_loader, params["criterion"](), yellowfin, device, params["view_every"])
      curr_acc = (test(model, val_loader, device)) * 100.0
      val_acc = curr_acc

      print(f"epoch {epoch + 1}/{params['epochs']}, Accuracy: {curr_acc:.4}")
    
    results[fold] = curr_acc

  # Print fold results
  print('--------------------------------')
  print(f'K-FOLD CROSS VALIDATION RESULTS FOR {params["folds"]} FOLDS')
  print('--------------------------------')
  
  sum = 0.0
  for fold, value in results.items():
    print(f"Fold {fold}: {value} %")
    sum += value
  
  avg_acc = sum/len(results.items())
  print(f"Average: {avg_acc:.4} %")
  print("-----------")
  
  return avg_acc

In [23]:
parameters_yellowfin = {
    "criterion": [torch.nn.CrossEntropyLoss],
    "lr": [0.002, 0.02, 0.2],
    "batch_size": [90, 120, 180],
    "folds": [5],
    "epochs": [25],
    "wd": [0],
    "mu": [0],
    "view_every": [1000]
   }

model = Net()
model.to(device)

opt_config_yellowfin, opt_acc_yellowfin = hyperparameters_yellowfin(model, parameters_yellowfin, device)

  0%|          | 0/9 [00:00<?, ?it/s]

-----------------------
Current Parameters:  {'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'lr': 0.002, 'batch_size': 90, 'folds': 5, 'epochs': 25, 'wd': 0, 'mu': 0, 'view_every': 1000}
---------------Fold 1--------------


/content/yellowfin.py:526: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self._var_list, np.sqrt(self._stat_protect_fac * self._h_max) + eps)
/content/yellowfin.py:555: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self._var_list, self._exploding_grad_clip_target_value + eps)


Batch 500 / 534: loss = 0.4508
epoch 1/25, Accuracy: 94.58
Batch 500 / 534: loss = 0.1405
epoch 2/25, Accuracy: 95.25
Batch 500 / 534: loss = 0.1076
epoch 3/25, Accuracy: 96.19
Batch 500 / 534: loss = 0.0950
epoch 4/25, Accuracy: 96.04
Batch 500 / 534: loss = 0.0893
epoch 5/25, Accuracy: 96.02
Batch 500 / 534: loss = 0.0968
epoch 6/25, Accuracy: 96.08
Batch 500 / 534: loss = 0.0987
epoch 7/25, Accuracy: 96.28
Batch 500 / 534: loss = 0.0983
epoch 8/25, Accuracy: 96.13
Batch 500 / 534: loss = 0.1132
epoch 9/25, Accuracy: 95.96
Batch 500 / 534: loss = 0.1046
epoch 10/25, Accuracy: 95.68
Batch 500 / 534: loss = 0.1167
epoch 11/25, Accuracy: 96.17
Batch 500 / 534: loss = 0.1149
epoch 12/25, Accuracy: 95.24
Batch 500 / 534: loss = 0.1147
epoch 13/25, Accuracy: 94.99
Batch 500 / 534: loss = 0.1237
epoch 14/25, Accuracy: 96.06
Batch 500 / 534: loss = 0.1405
epoch 15/25, Accuracy: 95.93
Batch 500 / 534: loss = 0.1347
epoch 16/25, Accuracy: 95.31
Batch 500 / 534: loss = 0.1517
epoch 17/25, Accur